# Pipeline, Validação e Treinamento

In [132]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import joblib
import mlflow
import os as os 

## Carregar Dataset CSV Limpo

In [133]:
caminho = "../data/tweets_limpo.csv"

if not os.path.exists(caminho):
    raise FileNotFoundError(f"Arquivo não encontrado: {caminho}")

df = pd.read_csv(caminho)
display(df.head())

,text,label,text_length
0,Adorei o produto de investimento no mercado de...,positivo,56
1,O serviço de instalacão da internet foi péssim...,negativo,62
2,Entrega rápida com atendimento excelente,positivo,40
3,Produto veio com defeito estou decepcionado,negativo,43
4,Gostei muito da qualidade,positivo,25


## Validações

In [134]:
assert df["text"].notnull().all(), "Coluna 'text' possui valores nulos."
assert df["label"].notnull().all(), "Coluna 'label' possui valores nulos."
assert df["text"].apply(lambda x: isinstance(x, str)).all(), "Coluna 'text' não é do tipo string."
assert df["label"].isin(["positivo", "negativo"]).all(), "Coluna 'label' possui valores inesperados."

print("Validação de dados concluída com sucesso!")

Validação de dados concluída com sucesso!


## Separação: Treino e Teste

In [135]:
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["label"], test_size=0.2, random_state=42)

In [136]:
X_train

3           Produto veio com defeito estou decepcionado
13                         Nao funcionou como esperado.
16                           Top esta carteira da marca
15                        Nossa este produto é nota 10!
11          Produto veio com defeito estou decepcionado
2              Entrega rápida com atendimento excelente
9     O serviço de instalacão da internet foi pessim...
17                Muito ruim, não gostei do atendimento
4                             Gostei muito da qualidade
12                            Gostei muito da qualidade
7                                Não voltarei a comprar
10             Entrega rapida com atendimento excelente
14                              Fui muito bem atendido.
6                               Fui muito bem atendido.
Name: text, dtype: object

In [137]:
X_test

0    Adorei o produto de investimento no mercado de...
1    O serviço de instalacão da internet foi péssim...
8    Adorei o produto de investimento no mercado de...
5                         Não funcionou como esperado.
Name: text, dtype: object

## Pipeline Exibição

In [138]:
pipeline = Pipeline(
    [
        ("vect", CountVectorizer()),
        ("clf", MultinomialNB()),
    ]
)
pipeline.fit(X_train, y_train)

,steps,"[('vect', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [139]:
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negativo       0.50      1.00      0.67         2
    positivo       0.00      0.00      0.00         2

    accuracy                           0.50         4
   macro avg       0.25      0.50      0.33         4
weighted avg       0.25      0.50      0.33         4



d:\temp\mlops_pipeline\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\temp\mlops_pipeline\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\temp\mlops_pipeline\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


## Salvar o modelo e o vetor

In [140]:
joblib.dump(pipeline.named_steps["clf"], "../model.joblib")
joblib.dump(pipeline.named_steps["vect"], "../vectorizer.joblib")

['../vectorizer.joblib']

## Rastrear os experimentos com MLFLOW

In [141]:
mlflow.set_experiment("sentiment_pipeline")
with mlflow.start_run():
    mlflow.log_param("modelo", "MultinomialNB")
    mlflow.log_param("vetor", "CountVectorizer")
    mlflow.sklearn.log_model(pipeline, "modelo_sentimento")

2025/06/28 12:19:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 12:20:04 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/06/28 12:20:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
